In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm

from fair21.energy_balance_model import EnergyBalanceModel

In [ ]:
# coordinates
timepoints = pd.date_range('2000-07-01', periods=50, freq='12MS')
timebounds = pd.date_range('2000-01-01', periods=51, freq='12MS')
timestep = 1
scenarios = ['abrupt', 'ramp']
configs = ['high', 'central', 'low']
species = ['CO2 fossil', 'CO2 AFOLU', 'CO2', 'CH4', 'N2O', 'Sulfur', 'Aerosol-radiation interactions', 'Aerosol-cloud interactions']
gasboxes = range(4)
layers = range(3)

In [ ]:
# these will be internal
n_scenarios = len(scenarios)
n_species = len(species)
n_configs = len(configs)
n_timepoints = len(timepoints)
n_timebounds = len(timebounds)
n_gasboxes = len(gasboxes)
n_layers = len(layers)

In [ ]:
# Create placeholder arrays. This would be inside the FAIR class
emissions = xr.DataArray(
    np.ones((n_timepoints, n_scenarios, n_configs, n_species)) * np.nan,
    coords = (timepoints, scenarios, configs, species),
    dims = ('timepoints', 'scenario', 'config', 'specie')
)
concentration = xr.DataArray(
    np.ones((n_timebounds, n_scenarios, n_configs, n_species)) * np.nan,
    coords = (timebounds, scenarios, configs, species),
    dims = ('timebounds', 'scenario', 'config', 'specie')
)
forcing = xr.DataArray(
    np.ones((n_timebounds, n_scenarios, n_configs, n_species)) * np.nan,
    coords = (timebounds, scenarios, configs, species),
    dims = ('timebounds', 'scenario', 'config', 'specie')
)
temperature = xr.DataArray(
    np.ones((n_timebounds, n_scenarios, n_configs, n_layers)) * np.nan,
    coords = (timebounds, scenarios, configs, layers),
    dims = ('timebounds', 'scenario', 'config', 'layer')
)

In [ ]:
# initialise configs
climate_configs = xr.Dataset(
    {
        'ocean_heat_transfer': (["config", "layer"], np.ones((n_configs, n_layers)) * np.nan),
        'ocean_heat_capacity': (["config", "layer"], np.ones((n_configs, n_layers)) * np.nan),
        'deep_ocean_efficacy': ("config", np.ones(n_configs) * np.nan),
        'stochastic_run': ("config", np.zeros(n_configs, dtype=bool)),
        'sigma_eta': ("config", np.ones(n_configs) * 0.5),
        'sigma_xi': ("config", np.ones(n_configs) * 0.5),
        'gamma_autocorrelation': ("config", np.ones(n_configs) * 2),#("config", np.ones(n_configs) * np.nan),
        'seed': ("config", np.zeros(n_configs, dtype=np.uint32)),
        'use_seed': ("config", np.zeros(n_configs, dtype=bool)),
    },
    coords = {
        "config": configs,
        "layer": layers
    },
)

In [ ]:
climate_configs

In [ ]:
# fill in configs
climate_configs["ocean_heat_transfer"].loc[dict(config='high')] = np.array([0.6, 1.3, 1.0])
climate_configs["ocean_heat_capacity"].loc[dict(config='high')] = np.array([5, 15, 80])
climate_configs["deep_ocean_efficacy"].loc[dict(config='high')] = 1.29

climate_configs["ocean_heat_transfer"].loc[dict(config='central')] = np.array([1.1, 1.6, 0.9])
climate_configs["ocean_heat_capacity"].loc[dict(config='central')] = np.array([8, 14, 100])
climate_configs["deep_ocean_efficacy"].loc[dict(config='central')] = 1.1

climate_configs["ocean_heat_transfer"].loc[dict(config='low')] = np.array([1.7, 2.0, 1.1])
climate_configs["ocean_heat_capacity"].loc[dict(config='low')] = np.array([6, 11, 75])
climate_configs["deep_ocean_efficacy"].loc[dict(config='low')] = 0.8

#climate_configs

In [ ]:
aci_parameters = ['scale', 'shape_Sulfur', 'shape_BC+OC']
n_aci_parameters = len(aci_parameters)

In [ ]:
# initialise species configs
species_configs = xr.Dataset(
    {
        # general parameters applicable to all species
        'tropospheric_adjustment': (["config", "specie"], np.zeros((n_configs, n_species))),
        'forcing_efficacy': (["config", "specie"], np.ones((n_configs, n_species))),
        'forcing_temperature_feedback': (["config", "specie"], np.zeros((n_configs, n_species))),
        'forcing_scale': (["config", "specie"], np.ones((n_configs, n_species))),
        
        # greenhouse gas parameters
        'partition_fraction': (
            ["config", "specie", "gasbox"], np.ones((n_configs, n_species, n_gasboxes)) * np.nan
        ),
        'unperturbed_lifetime': (
            ["config", "specie", "gasbox"], np.ones((n_configs, n_species, n_gasboxes)) * np.nan
        ),
        'molecular_weight': ("specie", np.ones(n_species) * np.nan),
        'baseline_concentration': (["config", "specie"], np.ones((n_configs, n_species)) * np.nan),
        'iirf_0': (["config", "specie"], np.ones((n_configs, n_species)) * np.nan),
        'iirf_airborne': (["config", "specie"], np.ones((n_configs, n_species)) * np.nan),
        'iirf_uptake': (["config", "specie"], np.ones((n_configs, n_species)) * np.nan),
        'iirf_temperature': (["config", "specie"], np.ones((n_configs, n_species)) * np.nan),
        'baseline_emissions': (["config", "specie"], np.zeros((n_configs, n_species))),
        
        # general parameters relating emissions, concentration or forcing of one species to forcing of another
        # these are all linear factors
        'contrails_radiative_efficiency': (["config", "specie"], np.zeros((n_configs, n_species))),
        'erfari_radiative_efficiency': (["config", "specie"], np.zeros((n_configs, n_species))),
        'h2o_stratospheric_factor': (["config", "specie"], np.zeros((n_configs, n_species))),
        'lapsi_radiative_efficiency': (["config", "specie"], np.zeros((n_configs, n_species))),
        'land_use_cumulative_emissions_to_forcing': (["config", "specie"], np.zeros((n_configs, n_species))),
        'ozone_radiative_efficiency': (["config", "specie"], np.zeros((n_configs, n_species))),
        
        # specific parameters for ozone-depleting GHGs
        'cl_atoms': ("specie", np.zeros(n_species)),
        'br_atoms': ("specie", np.zeros(n_species)),
        'fractional_release': (["config", "specie"], np.zeros((n_configs, n_species))),
        
#         # specific parameters for methane lifetime
#         'ch4_soil_lifetime'
#         'ch4_lifetime_chemical_sensitivity'
#         'ch4_lifetime_temperature_sensitivity'
#         'normalisation_2014_1850'

        # specific parameters for aerosol-cloud interactions
        'aci_parameters': (["config", "aci_parameter"], np.ones((n_configs, n_aci_parameters)) * np.nan)
            # n_aci_parameters can be defined at the top level
    },
    coords = {
        "config": configs,
        "specie": species,
        "gasbox": gasboxes,
        "aci_parameter": aci_parameters
    },
)

In [ ]:
species_configs['aci_parameters']

In [ ]:
# fill in species configs
species_configs["partition_fraction"].loc[dict(specie="CO2")] = np.array([0.2173, 0.2240, 0.2824, 0.2763])

non_co2_ghgs = ["CH4", "N2O"]
for gas in non_co2_ghgs:
    species_configs["partition_fraction"].loc[dict(specie=gas)] = np.array([1, 0, 0, 0])
    
species_configs["unperturbed_lifetime"].loc[dict(specie="CO2")] = np.array([1e9, 394.4, 36.54, 4.304])
species_configs["unperturbed_lifetime"].loc[dict(specie="CH4")] = 8.25
species_configs["unperturbed_lifetime"].loc[dict(specie="N2O")] = 109

species_configs["baseline_concentration"].loc[dict(specie="CO2")] = 278.3
species_configs["baseline_concentration"].loc[dict(specie="CH4")] = 729
species_configs["baseline_concentration"].loc[dict(specie="N2O")] = 270.3

species_configs["molecular_weight"].loc[dict(specie="CO2")] = 44.009
species_configs["molecular_weight"].loc[dict(specie="CH4")] = 16.043
species_configs["molecular_weight"].loc[dict(specie="N2O")] = 44.013

In [ ]:
# define one instance
species_configs["iirf_0"].loc[dict(specie="CO2", config="low")] = 30

In [ ]:
def calculate_iirf0(species_configs, iirf_horizon=100):
    gasbox_axis = species_configs["partition_fraction"].get_axis_num('gasbox')  # MAKE CONSTANT
    iirf_0 = (
        np.sum(species_configs["unperturbed_lifetime"] *
        (1 - np.exp(-iirf_horizon / species_configs["unperturbed_lifetime"]))
        * species_configs["partition_fraction"], gasbox_axis)
    )
    return iirf_0

In [ ]:
# fill in species configs
species_configs["partition_fraction"].loc[dict(specie="CO2")] = np.array([0.2173, 0.2240, 0.2824, 0.2763])

non_co2_ghgs = ["CH4", "N2O"]
for gas in non_co2_ghgs:
    species_configs["partition_fraction"].loc[dict(specie=gas)] = np.array([1, 0, 0, 0])
    
species_configs["unperturbed_lifetime"].loc[dict(specie="CO2")] = np.array([1e9, 394.4, 36.54, 4.304])
species_configs["unperturbed_lifetime"].loc[dict(specie="CH4")] = 8.25
species_configs["unperturbed_lifetime"].loc[dict(specie="N2O")] = 109

species_configs["baseline_concentration"].loc[dict(specie="CO2")] = 278.3
species_configs["baseline_concentration"].loc[dict(specie="CH4")] = 729
species_configs["baseline_concentration"].loc[dict(specie="N2O")] = 270.3

species_configs["molecular_weight"].loc[dict(specie="CO2")] = 44.009
species_configs["molecular_weight"].loc[dict(specie="CH4")] = 16.043
species_configs["molecular_weight"].loc[dict(specie="N2O")] = 44.013

In [ ]:
species_configs

In [ ]:
species_configs["iirf_0"] = calculate_iirf0(species_configs)  # will clobber existing value

In [ ]:
species_configs["iirf_0"]

In [ ]:
# override CO2 value as calculation is for present day
species_configs["iirf_0"].loc[dict(specie='CO2')] = 29

In [ ]:
species_configs["iirf_airborne"].loc[dict(specie='CO2', config='central')] = 0.000819
species_configs["iirf_airborne"].loc[dict(specie='CO2', config='low')] = 0
species_configs["iirf_airborne"].loc[dict(specie='CO2', config='high')] = 0.000819*2

In [ ]:
species_configs["iirf_airborne"].loc[dict(specie='CH4')] = 0.00032
species_configs["iirf_airborne"].loc[dict(specie='N2O')] = -0.0065

In [ ]:
species_configs["iirf_uptake"].loc[dict(specie='CO2', config='central')] = 0.00846
species_configs["iirf_uptake"].loc[dict(specie='CO2', config='low')] = 0
species_configs["iirf_uptake"].loc[dict(specie='CO2', config='high')] = 0.00846*2

In [ ]:
species_configs["iirf_uptake"].loc[dict(specie='CH4')] = 0
species_configs["iirf_uptake"].loc[dict(specie='N2O')] = 0

In [ ]:
species_configs["iirf_temperature"].loc[dict(specie='CO2', config='central')] = 4.0
species_configs["iirf_temperature"].loc[dict(specie='CO2', config='low')] = 0
species_configs["iirf_temperature"].loc[dict(specie='CO2', config='high')] = 8.0
species_configs["iirf_temperature"].loc[dict(specie='CH4')] = -0.3
species_configs["iirf_temperature"].loc[dict(specie='N2O')] = 0

In [ ]:
# define emissions
# TODO: write convenience functions to fill in config dimension
emissions.loc[dict(scenario='abrupt', specie='CO2 fossil')] = 38
emissions.loc[dict(scenario='abrupt', specie='CO2 AFOLU')] = 3
emissions.loc[dict(scenario='abrupt', specie='Sulfur')] = 100
emissions.loc[dict(scenario='ramp', specie='CO2 fossil')] = np.linspace(0, 38, 50)[:, None]
emissions.loc[dict(scenario='ramp', specie='CO2 AFOLU')] = np.linspace(0, 3, 50)[:, None]
emissions.loc[dict(scenario='ramp', specie='Sulfur')] = np.linspace(2.2, 100, 50)[:, None]

# TODO: automatic
emissions.loc[dict(specie='CO2')] = emissions.loc[dict(specie='CO2 fossil')] + emissions.loc[dict(specie='CO2 AFOLU')]

# fill in concentrations
concentration.loc[dict(scenario='abrupt', specie='CH4')] = 1800
concentration.loc[dict(scenario='abrupt', specie='N2O')] = 325
concentration.loc[dict(scenario='ramp', specie='CH4')] = np.linspace(729, 1800, 51)[:, None]
concentration.loc[dict(scenario='ramp', specie='N2O')] = np.linspace(270, 325, 51)[:, None]

In [ ]:
emissions

In [ ]:
concentration

In [ ]:
def multi_ebm(
    configs,
    ocean_heat_capacity,
    ocean_heat_transfer,
    deep_ocean_efficacy,
    stochastic_run,
    sigma_eta,
    sigma_xi,
    gamma_autocorrelation,
    seed,
    use_seed,
    forcing_4co2,
    n_timesteps
):
    """Create several instances of the EnergyBalanceModel for efficient implementation in FaIR.
    
    We have to use a for loop at is does not look like the linear algebra functions in scipy are naturally
    parallel.
    """
    
    n_runs = ocean_heat_capacity.shape[1]
    ebms = xr.Dataset(
        {
            "eb_matrix_d": (["config", "eb_dim0", "eb_dim1"], np.ones((n_configs, n_layers+1, n_layers+1))*np.nan),
            "forcing_vector_d": (["config", "eb_dim0"], np.ones((n_configs, n_layers+1))*np.nan),
            "stochastic_d": (["timebounds", "config", "eb_dim0"], np.ones((n_timebounds, n_configs, n_layers+1))*np.nan),
            "ecs": (["config"], np.ones(n_configs) * np.nan),
            "tcr": (["config"], np.ones(n_configs) * np.nan),
        },
        coords = {
            "timebounds": timebounds,
            "config": configs,
            "eb_dim0": np.arange(-1, n_layers),
            "eb_dim1": np.arange(-1, n_layers),
        }
    )
    
    for i_run, config in enumerate(configs):
        ebm = EnergyBalanceModel(
            ocean_heat_capacity=ocean_heat_capacity[i_run, :],
            ocean_heat_transfer=ocean_heat_transfer[i_run, :],
            deep_ocean_efficacy=deep_ocean_efficacy[i_run],
            stochastic_run=stochastic_run[i_run],
            sigma_eta=sigma_eta[i_run],
            sigma_xi=sigma_xi[i_run],
            gamma_autocorrelation=gamma_autocorrelation[i_run],
            seed=seed[i_run] if use_seed[i_run] else None,
            forcing_4co2=forcing_4co2[i_run],
            n_timesteps=n_timebounds,
        )
        ebms["eb_matrix_d"].loc[dict(config=config)]=ebm.eb_matrix_d
        ebms["forcing_vector_d"].loc[dict(config=config)]=ebm.forcing_vector_d
        ebms["stochastic_d"].loc[dict(config=config)]=ebm.stochastic_d
        ebm.emergent_parameters()
        ebms["ecs"].loc[dict(config=config)]=ebm.ecs
        ebms["tcr"].loc[dict(config=config)]=ebm.tcr
        
    return ebms

In [ ]:
ebms = multi_ebm(
    configs,
    ocean_heat_capacity=climate_configs['ocean_heat_capacity'],
    ocean_heat_transfer=climate_configs['ocean_heat_transfer'],
    deep_ocean_efficacy=climate_configs['deep_ocean_efficacy'],
    stochastic_run=climate_configs['stochastic_run'],
    sigma_eta=climate_configs['sigma_eta'],
    sigma_xi=climate_configs['sigma_xi'],
    gamma_autocorrelation=climate_configs['gamma_autocorrelation'],
    seed=climate_configs['seed'],
    use_seed=climate_configs['use_seed'],
    forcing_4co2=np.ones(3)*8,
    n_timesteps=n_timebounds,
)

In [ ]:
ebms["eb_matrix_d"]

In [ ]:
ebms["forcing_vector_d"]

In [ ]:
ebms["stochastic_d"]

In [ ]:
ebms["ecs"]

In [ ]:
ebms["tcr"]

In [ ]:
# Define first timestep
# TODO: make convenience function
concentration.loc[dict(specie='CO2', timebounds='2000-01-01')] = 278.3

In [ ]:
cumulative_emissions = xr.DataArray(
    np.ones((n_timebounds, n_scenarios, n_configs, n_species)) * np.nan,
    coords = (timebounds, scenarios, configs, species),
    dims = ('timebounds', 'scenario', 'config', 'specie')
)

In [ ]:
cumulative_emissions[dict(timebounds=0)] = 0   # initial condition

In [ ]:
cumulative_emissions[dict(timebounds=slice(1,None))] = emissions.cumsum(axis=0, skipna=False) * timestep

In [ ]:
# as part of species configs

def calculate_g(species_configs, iirf_horizon=100):
    gasbox_axis = species_configs["partition_fraction"].get_axis_num('gasbox')
    g1 = np.sum(
        species_configs["partition_fraction"] * species_configs["unperturbed_lifetime"] *
        (1 - (1 + iirf_horizon/species_configs["unperturbed_lifetime"]) *
        np.exp(-iirf_horizon/species_configs["unperturbed_lifetime"])),
    axis=gasbox_axis)
    g0 = np.exp(-1 * np.sum((species_configs["partition_fraction"])*
        species_configs["unperturbed_lifetime"]*
        (1 - np.exp(-iirf_horizon/species_configs["unperturbed_lifetime"])), axis=gasbox_axis)/
        g1
    )
    return g0, g1

def append_g(species_configs):
    g0, g1 = calculate_g(species_configs, iirf_horizon=100)
    species_configs["g0"]=g0
    species_configs["g1"]=g1
    return species_configs

In [ ]:
# as part of species configs

def calculate_concentration_per_emission(species_configs, mass_atmosphere=5.1352e18, molecular_weight_air=28.97):
    concentration_per_emission = 1 / (
        mass_atmosphere / 1e18 * 
        species_configs["molecular_weight"] / molecular_weight_air
    )
    return concentration_per_emission
    
def append_concentration_per_emission(species_configs):
    species_configs["concentration_per_emission"] = calculate_concentration_per_emission(species_configs)
    return species_configs

In [ ]:
species_configs = append_g(species_configs)
species_configs = append_concentration_per_emission(species_configs)

In [ ]:
species_configs

In [ ]:
airborne_emissions = xr.DataArray(
    np.zeros((n_timebounds, n_scenarios, n_configs, n_species)),
    coords = (timebounds, scenarios, configs, species),
    dims = ('timebounds', 'scenario', 'config', 'specie')
)

alpha_lifetime = xr.DataArray(
    np.ones((n_timebounds, n_scenarios, n_configs, n_species)) * np.nan,
    coords = (timebounds, scenarios, configs, species),
    dims = ('timebounds', 'scenario', 'config', 'specie')
)

In [ ]:
# gas_partitions = xr.DataArray(
#     np.zeros((n_scenarios, n_configs, n_species, n_gasboxes)),
#     coords = (scenarios, configs, species, gasboxes),
#     dims = ('scenario', 'config', 'specie', 'gasbox')
# )
gas_partitions_array = np.zeros((n_scenarios, n_configs, n_species, n_gasboxes))

In [ ]:
gas_partitions_array

In [ ]:
# defined at top level
species_in = {
    'CO2 fossil': {
        'emissions': True,
        'concentration': False,
        'forcing': False,
        'input': 'emissions',
        'greenhouse_gas': False,  # it doesn't behave as a GHG in the model
    },
    'CO2 AFOLU': {
        'emissions': True,
        'concentration': False,
        'forcing': False,
        'input': 'emissions',
        'greenhouse_gas': False,  # it doesn't behave as a GHG in the model
    },
    'CO2': {
        'emissions': True,
        'concentration': True,
        'forcing': True,
        'input': 'emissions',
        'greenhouse_gas': True,
    },
    'CH4': {
        'emissions': True,
        'concentration': True,
        'forcing': True,
        'input': 'concentration',
        'greenhouse_gas': True,
    },
    'N2O': {
        'emissions': True,
        'concentration': True,
        'forcing': True,
        'input': 'concentration',
        'greenhouse_gas': True,
    },
    'Sulfur': {
        'emissions': True,
        'concentration': False,
        'forcing': True,
        'input': 'emissions',
        'greenhouse_gas': False,
    },
    'Aerosol-radiation interactions': {
        'emissions': False,
        'concentration': False,
        'forcing': True,
        'input': 'calculated',
        'greenhouse_gas': False,
    },
    'Aerosol-cloud interactions': {
        'emissions': False,
        'concentration': False,
        'forcing': True,
        'input': 'calculated',
        'greenhouse_gas': False,
    }
}

In [ ]:
species_in_df = pd.DataFrame(
    species_in
)
species_in_df

In [ ]:
species_in_df.loc['emissions']

In [ ]:
emissions_species = list(species_in_df.loc['emissions',species_in_df.loc['emissions']==True].index)
concentration_species = list(species_in_df.loc['concentration',species_in_df.loc['concentration']==True].index)
forcing_species = list(species_in_df.loc['forcing',species_in_df.loc['forcing']==True].index)
greenhouse_gas_species = list(species_in_df.loc['greenhouse_gas',species_in_df.loc['greenhouse_gas']==True].index)

In [ ]:
emissions_species

In [ ]:
np.asarray(((species_in_df.loc['input']=='emissions')&(species_in_df.loc['greenhouse_gas'])).values, dtype=bool)

In [ ]:
emissions_indices = np.asarray(species_in_df.loc['emissions'].values, dtype=bool)
concentration_indices = np.asarray(species_in_df.loc['concentration'].values, dtype=bool)
forcing_indices = np.asarray(species_in_df.loc['forcing'].values, dtype=bool)
greenhouse_gas_forward_indices = np.asarray(
    (
        (species_in_df.loc['input']=='emissions')&
        (species_in_df.loc['greenhouse_gas'])
    ).values,
    dtype=bool
)

In [ ]:
emissions_indices
greenhouse_gas_forward_indices

In [ ]:
# emissions_array = emissions.data
# type(emissions_array)

In [ ]:
concentration_array = concentration.data
forcing_array = forcing.data
temperature_array = temperature.data

In [ ]:
emissions[:,:,:,greenhouse_gas_forward_indices]

In [ ]:
# array function - not xarray

import warnings

def calculate_alpha(
    airborne_emissions,
    cumulative_emissions,
    g0,
    g1,
    iirf_0,
    iirf_airborne,
    iirf_temperature,
    iirf_uptake,
    temperature,
    iirf_max=100,
):

    iirf = iirf_0 + iirf_uptake * (cumulative_emissions-airborne_emissions) + iirf_temperature * temperature + iirf_airborne * airborne_emissions
    iirf = (iirf>iirf_max) * iirf_max + iirf * (iirf<iirf_max)
    # overflow and invalid value errors occur with very large and small values
    # in the exponential. This happens with very long lifetime GHGs. Usually
    # these GHGs don't have a temperature dependence on IIRF but even if they
    # did the lifetimes are so long that it is unlikely to have an effect.
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        alpha = g0 * np.exp(iirf / g1)
#        alpha[np.isnan(alpha)]=1
    return alpha

In [ ]:
# array function

def step_concentration(
    emissions,
    gas_boxes_old,
    airborne_emissions_old,
    alpha_lifetime,
    baseline_concentration,
    baseline_emissions,
    concentration_per_emission,
    gasbox_axis,
    lifetime,
    partition_fraction,
    timestep,
):

    decay_rate = timestep/(alpha_lifetime * lifetime)
    decay_factor = np.exp(-decay_rate)

    # additions and removals
    gas_boxes_new = (
        partition_fraction *
        (emissions - baseline_emissions) *
        1 / decay_rate *
        (1 - decay_factor) * timestep + gas_boxes_old * decay_factor
    )

    airborne_emissions_new = np.sum(gas_boxes_new, axis=gasbox_axis)
    concentration_out = baseline_concentration + concentration_per_emission * airborne_emissions_new
    
    return concentration_out, gas_boxes_new, airborne_emissions_new

In [ ]:
alpha_lifetime.shape

In [ ]:
#concentration[i_timepoint+1, ..., greenhouse_gas_forward_indices].shape

In [ ]:
airborne_emissions.shape

In [ ]:
%%time
alpha_lifetime_array = alpha_lifetime.data
airborne_emissions_array = airborne_emissions.data
cumulative_emissions_array = cumulative_emissions.data
g0_array = species_configs['g0'].data
g1_array = species_configs['g1'].data
iirf_0_array = species_configs['iirf_0'].data
iirf_airborne_array = species_configs['iirf_airborne'].data
iirf_temperature_array = species_configs['iirf_temperature'].data
iirf_uptake_array = species_configs['iirf_uptake'].data
iirf_temperature = species_configs['iirf_temperature'].data
concentration_array = concentration.data
#gas_partitions_array = gas_partitions.data
airborne_emissions_array = airborne_emissions.data
emissions_array = emissions.data
baseline_concentration_array = species_configs['baseline_concentration'].data
baseline_emissions_array = species_configs['baseline_emissions'].data
concentration_per_emission_array = species_configs['concentration_per_emission'].data
unperturbed_lifetime_array = species_configs['unperturbed_lifetime'].data
partition_fraction_array = species_configs['partition_fraction'].data

In [ ]:
#greenhouse_gas_forward_indices = np.array([True, False, True, False, True, False, True, False])

In [ ]:
#alpha_lifetime_array[i_timepoint:i_timepoint+1, ..., greenhouse_gas_forward_indices].shape

In [ ]:
# airborne_emissions_array[i_timepoint:i_timepoint+1, ..., greenhouse_gas_forward_indices].shape
# #cumulative_emissions_array.shape
# g0_array[None, None, ..., greenhouse_gas_forward_indices].shape
# iirf_0_array[None, None, ..., greenhouse_gas_forward_indices].shape

In [ ]:
# concentration_array[i_timepoint+1:i_timepoint+2, ..., greenhouse_gas_forward_indices].shape
# gas_partitions_array[None, ..., greenhouse_gas_forward_indices, :].shape
# airborne_emissions[i_timepoint+1:i_timepoint+2, ..., greenhouse_gas_forward_indices].shape
# emissions_array[i_timepoint:i_timepoint+1, ..., greenhouse_gas_forward_indices, None].shape
# baseline_concentration_array[None, None, ..., greenhouse_gas_forward_indices].shape
# #unperturbed_lifetime_array.shape

In [ ]:
IIRF_MAX = 100
gasbox_axis = species_configs["partition_fraction"].get_axis_num('gasbox')

for i_timepoint in tqdm(range(n_timepoints)):  # 0 to 751
    # i think we want alpha in all GHG cases?
    alpha_lifetime_array[i_timepoint:i_timepoint+1, ..., greenhouse_gas_forward_indices] = calculate_alpha(   # this timepoint
        airborne_emissions_array[i_timepoint:i_timepoint+1, ..., greenhouse_gas_forward_indices],  # last timebound
        cumulative_emissions_array[i_timepoint:i_timepoint+1, ..., greenhouse_gas_forward_indices],  # last timebound
        g0_array[None, None, ..., greenhouse_gas_forward_indices],
        g1_array[None, None, ..., greenhouse_gas_forward_indices],
        iirf_0_array[None, None, ..., greenhouse_gas_forward_indices],
        iirf_airborne_array[None, None, ..., greenhouse_gas_forward_indices],
        iirf_temperature_array[None, None, ..., greenhouse_gas_forward_indices],
        iirf_uptake_array[None, None, ..., greenhouse_gas_forward_indices],
        0,  # temperature here # last timebound
        IIRF_MAX
    )
    (
        concentration_array[i_timepoint+1:i_timepoint+2, ..., greenhouse_gas_forward_indices], 
        gas_partitions_array[..., greenhouse_gas_forward_indices, :], 
        airborne_emissions_array[i_timepoint+1:i_timepoint+2, ..., greenhouse_gas_forward_indices]
    ) = step_concentration( 
        emissions_array[i_timepoint:i_timepoint+1, ..., greenhouse_gas_forward_indices, None],  # this timepoint
        gas_partitions_array[..., greenhouse_gas_forward_indices, :], # last timebound
        airborne_emissions_array[i_timepoint+1:i_timepoint+2, ..., greenhouse_gas_forward_indices, None],  # last timebound
        alpha_lifetime_array[i_timepoint:i_timepoint+1, ..., greenhouse_gas_forward_indices, None],
        baseline_concentration_array[None, None, ..., greenhouse_gas_forward_indices],
        baseline_emissions_array[None, None, ..., greenhouse_gas_forward_indices, None],
        concentration_per_emission_array[None, None, ..., greenhouse_gas_forward_indices],
        4,
        unperturbed_lifetime_array[None, None, ..., greenhouse_gas_forward_indices, :],
#        oxidation_matrix,
        partition_fraction_array[None, None, ..., greenhouse_gas_forward_indices, :],
        timestep,
    )

In [ ]:
alpha_lifetime_array[:, 0, 0, 2]

In [ ]:
cumulative_emissions[i_timepoint, ..., greenhouse_gas_forward_indices]

In [ ]:
species_configs['g0'][..., greenhouse_gas_forward_indices]

In [ ]:
alpha_lifetime[i_timepoint, ..., greenhouse_gas_forward_indices].shape

In [ ]:
alpha_lifetime.shape

In [ ]:
airborne_emissions[i_timepoint, ..., greenhouse_gas_forward_indices].shape

In [ ]:
cumulative_emissions[i_timepoint, ..., greenhouse_gas_forward_indices].shape

In [ ]:
species_configs['iirf_airborne']#.shape

In [ ]:
species_configs['unperturbed_lifetime']

In [ ]:
alpha_lifetime[i_timepoint, ..., greenhouse_gas_forward_indices].expand_dims(dim={'gasbox':n_gasboxes}, axis=3).shape

In [ ]:
gasbox_axis

In [ ]:
alpha_lifetime[0,:,:,:]

In [ ]:
class FAIR:
    
    def __init__(self):
        pass
    
    def define_time(self, timepoints=None, timebounds=None):
        pass
    
    def define_scenarios(
        self,
        emissions=None,
        concentration=None,
        forcing=None,
        temperature=None
    ):

        # do some checking on the inputs
        self.emissions_array = emissions.data
        self.concentration_array = concentration.data
        self.forcing_array = forcing.data
        self.temperature_array = temperature.data
        

In [ ]:
emissions_species

In [ ]:
species_xr = xr.DataArray(species, dims=("specie",))
species_xr.isin(greenhouse_gas_species)

In [ ]:
concentration.data[:,0,0,4]